<a href="https://colab.research.google.com/github/frasheed-dev/edx_TinyML_specialization/blob/master/colabs/2-02-07-ExploringCategorical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start with a simple neural network for MNIST
MNIST details: https://en.wikipedia.org/wiki/MNIST_database

Nice blog: https://machinelearningmastery.com/how-to-normalize-center-and-standardize-images-with-the-imagedatagenerator-in-keras/


Note that there are 2 layers, one with 20 neurons, and one with 10.

The 10-neuron layer is our final layer because we have 10 classes we want to classify (0 to 9 digits).

Train this, and you should see it get about 98% accuracy

In [1]:
import tensorflow as tf
data = tf.keras.datasets.mnist

(training_images, training_labels), (val_images, val_labels) = data.load_data()

# normalizing images (pixel values are from 0 to 255, we are rescaling them from 0 to 1 here)
training_images  = training_images / 255.0
val_images = val_images / 255.0

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)), #Flatten: to flatten the input shape 
                                    tf.keras.layers.Dense(20, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=20, use_multiprocessing=True, validation_data=(val_images, val_labels))


11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/20
1875/1875 [==============================] - 8s 3ms/step - loss: 0.4102 - accuracy: 0.8870 - val_loss: 0.2697 - val_accuracy: 0.9227
Epoch 2/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2525 - accuracy: 0.9279 - val_loss: 0.2304 - val_accuracy: 0.9341
Epoch 3/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2179 - accuracy: 0.9382 - val_loss: 0.2090 - val_accuracy: 0.9364
Epoch 4/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1936 - accuracy: 0.9450 - val_loss: 0.1919 - val_accuracy: 0.9436
Epoch 5/20
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1738 - accuracy: 0.9503 - val_loss: 0.1808 - val_accuracy: 0.9461
Epoch 6/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1592 - accuracy: 0.9546 - val_loss: 0.1786 - val_accuracy: 0.9482
Epoch 7/20
1875/1875 [==============================] - 5s 3ms/step - l

## Examine the test data

Using model.evaluate, you can get metrics for a test set. In this case we only have a training set and a validation set, so we can try it out with the validation set. The accuracy will be slightly lower, at maybe 96%.  This is because the model hasn't previously seen this data and may not be fully generalized for all data. Still it's a pretty good score.


In [6]:
model.evaluate(val_images, val_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.1610 - accuracy: 0.9563


[0.1609831601381302, 0.9563000202178955]

You can also predict images, and compare against their actual label. The [0] image in the set is a number 7, and here you can see that neuron 7 has a 9.9e-1 (99%+) probability, so it got it right!

In [7]:
classifications = model.predict(val_images)
print(classifications[0])
print(val_labels[0])

[2.5342740e-06 6.2306458e-09 3.9046060e-07 5.2083802e-04 2.0226295e-12
 1.4485498e-08 6.5611205e-17 9.9947530e-01 5.3363163e-07 4.4881389e-07]
7


## Modify to inspect learned values

This code is identical, except that the layers are named prior to adding to the sequential. This allows us to inspect their learned parameters later.

In [8]:
import tensorflow as tf
data = tf.keras.datasets.mnist

(training_images, training_labels), (val_images, val_labels) = data.load_data()

training_images  = training_images / 255.0
val_images = val_images / 255.0
layer_1 = tf.keras.layers.Dense(20, activation=tf.nn.relu)
layer_2 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    layer_1,
                                    layer_2])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images, training_labels, epochs=20)

model.evaluate(val_images, val_labels)

classifications = model.predict(val_images)
print(classifications[0])
print(val_labels[0])

Epoch 1/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4020 - accuracy: 0.8877
Epoch 2/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2135 - accuracy: 0.9383
Epoch 3/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1814 - accuracy: 0.9470
Epoch 4/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1617 - accuracy: 0.9531
Epoch 5/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1489 - accuracy: 0.9567
Epoch 6/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1386 - accuracy: 0.9594
Epoch 7/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1307 - accuracy: 0.9622
Epoch 8/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1227 - accuracy: 0.9638
Epoch 9/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1172 - accuracy: 0.9653
Epoch 10/20
1875/1875 [==============================] - 5s 3ms/step - loss: 0.113

# Inspect weights

If you print layer_1.get_weights(), you'll see a lot of data. Let's unpack it. First, there are 2 arrays in the result, so let's look at the first one. In particular let's look at its size.

In [9]:
print(layer_1.get_weights()[0].size)

15680


The above code should print 15680. Why?

Recall that there are 20 neurons in the first layer.

Recall also that the images are 28x28, which is 784.

If you multiply 784 x 20 you get 15680.

So...this layer has 20 neurons, and each neuron learns a w parameter for each pixel. So instead of y=wx+b, we have 

y=w1X1 + w2X2 + w3X3 +....+ w784X784 + b in every neuron! 

Every pixel has a weight in every neuron. Those weights are multiplied by the pixel value, summed up, and given a bias.


In [11]:
print(layer_1.get_weights()[1].size)

20


The above code will give you 20 -- the get_weights()[1] contains the biases for each of the 20 neurons in this layer.

## Inspecting layer 2

Now let's look at layer 2. Printing the get_weights will give us 2 lists, the first a list of weights for the 10 neurons, and the second a list of biases for the 10 neurons

Let's look first at the weights:

In [12]:
print(layer_2.get_weights()[0].size)

200


This should return 200. Again, consider why?

There are 10 neurons in this layer, but there are 20 neurons in the previous layer. So, each neuron in this layer will learn a weight for the incoming value from the previous layer. So, for example, the if the first neuron in this layer is N21, and the neurons output from the previous layers are N1_1-N1_20, then this neuron will have 20 weights (w1-w20) and it will calculate its output to be:

neuron output (y) = w1N1_1 + w2N1_2 + w3N1_3 +...+ w20N1_20 + b

So each of these weights will be learned as will the bias, for every neuron.

Note that N1_1 refers to Layer 1 Neuron 1.


In [13]:
print(layer_2.get_weights()[1].size)

10


...and as expected there are 10 elements in this array, representing the 10 biases for the 10 neurons.

Hopefully this helps you see how the element of a simple neuron containing y=wx+b can be expanded greatly into a deep neural network, and that DNN can learn the parameters that match the 784 pixels of an image to their output!